![menorah on mantle](media/menorah_banner.JPG)

# Day 7 AoC

🕎 [Day 7 description](https://adventofcode.com/2022/day/7) 🕎


## Setup

In [ ]:
# imports
import os, re, sys, IPython, itertools

In [ ]:
# common helper, data import
def ans(val):
    return IPython.display.Markdown("**Answer: {}**".format(val))

data_fd = open('inputs/input-aoc-22-07.txt', 'r')
data = data_fd.read().strip().split('\n')

In [ ]:
class TreeIt(object):
    def __init__(self, tree):
        self._root = tree
        self.stack = [self._root]
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if len(self.stack) == 0:
            raise StopIteration
        result = self.stack.pop()
        if isinstance(result, AoCDir):
            self.stack.extend(result.children())
        return result
        

class Node(object):
    def __init__(self, parent, size=0, name=None):
        self._parent = parent
        self._size = size
        self._name = name
    def parent(self):
        return self._parent
        
    def size(self):
        return self._size
    
    def name(self):
        return self._name
    
    def __iter__(self):
        return TreeIt(self)
    
    def __repr__(self):
        return "{} : {}".format(self._name, self.size())
    
class AoCDir(Node):
    def __init__(self, parent, size=0, name=None):
        super().__init__(parent, size, name)
        self._children = []
        
    def add_child(self, childnode):
        self._children.append(childnode)
        
    def find_child(self, name):
        for c in self._children:
            if c.name() == name:
                return c
    def children(self):
        return self._children[:]
        
    def size(self):
        return sum([x.size() for x in self._children])
    

        

In [ ]:
def create_tree(data):
    root = AoCDir(None, name='/')
    current_dir = root
    for line in data[1:]:
        line = line.strip()
        if line.startswith("$ cd .."):
            current_dir = current_dir.parent()
        elif line.startswith("$ cd"):
            current_dir = current_dir.find_child(line[5:])
        elif line.startswith("$ ls"):
            pass
        elif line.startswith("dir"):
            current_dir.add_child(AoCDir(current_dir, name=line[4:]))
        elif line[0] in '0123456789':
            size = int(line.split(' ')[0])
            name = line.split(' ')[1]
            current_dir.add_child(Node(current_dir, size=size, name=name))
        else:
            print("Bad output {}".format(line))
    return root

## Part 1

In [ ]:
ans(sum([ x.size() for x in create_tree(data) if x.size() < 100000 and isinstance(x, AoCDir)]))

## Part 2

In [ ]:
root = create_tree(data)
needed = 30000000
total = 70000000
current = total - root.size()
tobefreed = needed - current
print(current)
print(tobefreed)

In [ ]:
options = [ x.size() for x in root if x.size() > tobefreed and isinstance(x, AoCDir)]
options.sort()
ans(options[0])

## Notes

Fair bit of code, but not really difficult. My biggest bug was using `size_` instead of `size()` somewhere. Had to remind myself of how to implement a python iterator. Also sorting Node objects by size didn't work, but we didn't actually need the name, so I just pulled out the size before sorting.